## Import Librarys

In [18]:
import numpy as np
import os
import cv2
import time
from model.yolo_model import YOLO

## Create some important functions

**Define the process image function**

In [25]:
def process_image(img):
    # resize
    image = cv2.resize(img, (416, 416), interpolation=cv2.INTER_CUBIC)
    
    image = np.array(image, dtype="float32")
    
    # normalize
    image /= 255.
    
    image = np.expand_dims(image, axis=0)
    
    return image

**Draw rectangle and detect function on image**

In [27]:
def draw(image, boxes, scores, classes, all_classes):
    
    for box, score, cl in zip(boxes, scores, classes):
        
        x, y, w, h = box
        
        top = max(0, np.floor(x+0.5).astype(int))
        left = max(0, np.floor(y+0.5).astype(int))
        
        right = min(image.shape[1], np.floor(x+w+0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y+h+0.5).astype(int))
        
        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)
        print("box coordinate x, y, w, h: {0}".format(box))
        
    print()
    

In [5]:
def detect_image(image, yolo, all_classes):
    
    pimage = process_image(image)
    
    start = time.time()
    
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    
    end = time.time()
    
    print("time: {0:.2f}".format(end-start))
    
    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)
        
    return image

In [9]:
def detect_video(video, yolo, all_classes):
    
    video_path = os.path.join("/Users/neemiasbsilva/Downloads/Computer-Vision-with-Python/06-Deep-Learning-Computer-Vision/06-YOLOv3/videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("/Users/neemiasbsilva/Downloads/Computer-Vision-with-Python/06-Deep-Learning-Computer-Vision/06-YOLOv3/videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()

**Get classes name of coco dataset**

In [11]:
def get_classes(file):
    
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

## Detecting image

In [22]:
yolo = YOLO(0.6, 0.5)
file = "coco_classes.txt"
all_classes = get_classes(file)

/Users/neemiasbsilva/venv/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [30]:
f = "person-car-01.jpg"
path = "images/"+f
image = cv2.imread(path)
print(image.shape)
image = detect_image(image, yolo, all_classes)
cv2.imwrite("images/result/"+f, image)

(427, 640, 3)
time: 6.71
box coordinate x, y, w, h: [ 21.1702919  235.38516712 430.62110901 191.63832432]



True